In [37]:
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\evani\AppData\Local\Temp\ipykernel_3032\2652246010.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [28]:
c = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C']\request_9.csv")

In [41]:
Cc = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c']\request_9.csv")

In [43]:
CcE = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E']\request_9.csv")

In [42]:
CcEe = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E', 'e']\request_9.csv")

In [35]:
CcEeK = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E', 'e', 'K']\request_9.csv")

In [45]:
days = len(c.reward)

In [46]:
c.reward.sum()/days

-1527.5310344827585

In [47]:
Cc.reward.sum()/days

-1479.2247540229887

In [48]:
CcE.reward.sum()/days

-1574.9282873563218

In [50]:
CcEe.reward.sum()/days

-1687.0244098850574

In [49]:
CcEeK.reward.sum()/days

-1710.7221657471266